In [ ]:
import os
from models import get_pcam_generators, simple_vit, model_preparation, train_model, evaluate_model

# assuming this file is in the 'code' directory, which is in the same folder as the 'Data' directory:


#         |---- code-- cnn.py
# parent--|                       |-- train
#         |---- Data-- train+val--|
#         |                       |-- validation
#         |---- logs
#         |
#         |---- metadata

# change 'dir' to your parent directory if the lines below don't work
os.chdir('..')
dir=os.getcwd()

c:\Users\20222787\.conda\envs\tf\lib\site-packages\tensorflow\python\keras\engine\training_arrays_v1.py:37: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.21.5)
  from scipy.sparse import issparse  # pylint: disable=g-import-not-at-top


In [ ]:


train_gen, val_gen = get_pcam_generators(dir+'\Data')

input_shape=(96,96,3)
model = simple_vit(input_shape=input_shape)

model_name='simple_vit2'
model, callbacks_list=model_preparation(model,model_name,dir)

epochs=5
model, history = train_model(model, train_gen, val_gen, epochs, callbacks_list)


Found 144000 images belonging to 2 classes.
Found 16000 images belonging to 2 classes.
Epoch 1/5
 607/4500 [===>..........................] - ETA: 19:02 - loss: 1.4030 - accuracy: 0.4975

In [ ]:
accuracy, recall, roc_auc, confusion_matrix = evaluate_model(model, val_gen)